# Chronic Care Cost Benchmarking


## 0. Life cycle of Machine learning Project

- Understanding the Problem Statement
- Data Collection
- Data Checks to perform
- Exploratory data analysis
- Data Pre-Processing
- Model Training
- Choose best model

## 1. Problem statement


## Introduction
The UHC Network Affordability team is focused on getting our members the right care for the best possible price in the most convenient manner. The task at hand is to perform an exploratory data analysis (EDA) on publicly available claim data for Medicare members in the US. Medicare is the government healthcare program for US citizens aged 65+.

The aim is to understand how the cost of treating certain chronic conditions varies across different providers.

## Data
The data is available on the CMS (Medicare) website:

- **Member Benefit Data**: [Download](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/DE1_0_2009_Beneficiary_Summary_File_Sample_20.zip)
- **Outpatient Claim Data**: [Download](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/DE1_0_2008_to_2010_Outpatient_Claims_Sample_20.zip)
- **User Documentation**: [Download](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/SynPUF_DUG.pdf)
- **Additional Info**: [CMS SynPUF Page](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs)

## Data Cleaning
- Convert the beneficiary summary file’s chronic illness columns (Boolean fields) into a single categorical variable, concatenating multiple true diagnoses.
- If a member has three or more chronic conditions, categorize these as “Multiple”.
- Join claims & benefit data.

## Basic Summaries
- What is the distribution of races?
- What is the most common chronic illness combination?
- Which chronic illness combination has the total highest cost?
- Which chronic illness combination has the highest cost per member?

## Benchmarking
The aim here is to understand the distribution of cost across providers treating members with these chronic illnesses. Benchmarking providers across types of care is often a helpful starting point to begin working with areas of excessive cost.

- For each provider (use `AT_PHYSN_NPI`) & chronic illness, calculate the cost per member.
- For each chronic illness combination, represent the distribution of costs per provider.
- How does this change if we filter out cases where a given Chronic Illness & Provider NPI combination only has one member?
- Which providers are consistently expensive across chronic illnesses they treat?

## Conclusion
During the interview, please be prepared to discuss the methodology, findings, and any challenges faced during the analysis.

## Limitations

- TODO: Draft:
    - The data is part of the synthethic sub sample from a set of 20 sub samples
    - This data is generated from 5% random sample of Medicare beneficiaries
    - File One: Beneficiaries from 2009
    - File Two: Outpatient data years 2008-2010

## 2. Import Data and Required Packages

####  Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set display options to show all columns
pd.set_option('display.max_columns', None)

#### Import the CSV Data as Pandas DataFrame

In [2]:
beneficiary_df = pd.read_csv('data\DE1_0_2009_Beneficiary_Summary_File_Sample_20.csv')
outpatient_claims_df = pd.read_csv('data\DE1_0_2008_to_2010_Outpatient_Claims_Sample_20.csv')

#### Show Top 5 Records

In [3]:
# Define a dictionary for renaming columns
benef_rename_dict = {
    'DESYNPUF_ID': 'Beneficiary_ID',
    'BENE_BIRTH_DT': 'Date_of_Birth',
    'BENE_DEATH_DT': 'Date_of_Death',
    'BENE_SEX_IDENT_CD': 'Sex',
    'BENE_RACE_CD': 'Race_Code',
    'BENE_ESRD_IND': 'ESRD_Indicator',
    'SP_STATE_CODE': 'State_Code',
    'BENE_COUNTY_CD': 'County_Code',
    'BENE_HI_CVRAGE_TOT_MONS': 'Coverage_Part_A_Months',
    'BENE_SMI_CVRAGE_TOT_MONS': 'Coverage_Part_B_Months',
    'BENE_HMO_CVRAGE_TOT_MONS': 'Coverage_HMO_Months',
    'PLAN_CVRG_MOS_NUM': 'Coverage_Part_D_Plan_Months',
    'SP_ALZHDMTA': 'Chronic_Alzheimer_Disease',
    'SP_CHF': 'Chronic_Heart_Failure',
    'SP_CHRNKIDN': 'Chronic_Kidney_Disease',
    'SP_CNCR': 'Chronic_Cancer',
    'SP_COPD': 'Chronic_Obstructive_Pulmonary_Disease',
    'SP_DEPRESSN': 'Chronic_Depression',
    'SP_DIABETES': 'Chronic_Diabetes',
    'SP_ISCHMCHT': 'Chronic_Ischemic_Heart_Disease',
    'SP_OSTEOPRS': 'Chronic_Osteoporosis',
    'SP_RA_OA': 'Chronic_Rheumatoid_Arthritis_Osteoarthritis',
    'SP_STRKETIA': 'Chronic_Stroke_TIA',
    'MEDREIMB_IP': 'Inpatient_Medical_Reimbursement',
    'BENRES_IP': 'Inpatient_Beneficiary_Responsibility',
    'PPPYMT_IP': 'Inpatient_Primary_Payer_Reimbursement',
    'MEDREIMB_OP': 'Outpatient_Medical_Reimbursement',
    'BENRES_OP': 'Outpatient_Beneficiary_Responsibility',
    'PPPYMT_OP': 'Outpatient_Primary_Payer_Reimbursement',
    'MEDREIMB_CAR': 'Carrier_Medical_Reimbursement',
    'BENRES_CAR': 'Carrier_Beneficiary_Responsibility',
    'PPPYMT_CAR': 'Carrier_Primary_Payer_Reimbursement'
}

# Rename columns using the dictionary
beneficiary_df.rename(columns=benef_rename_dict, inplace=True)

In [4]:
beneficiary_df.head()

,Beneficiary_ID,Date_of_Birth,Date_of_Death,Sex,Race_Code,ESRD_Indicator,State_Code,County_Code,Coverage_Part_A_Months,Coverage_Part_B_Months,Coverage_HMO_Months,Coverage_Part_D_Plan_Months,Chronic_Alzheimer_Disease,Chronic_Heart_Failure,Chronic_Kidney_Disease,Chronic_Cancer,Chronic_Obstructive_Pulmonary_Disease,Chronic_Depression,Chronic_Diabetes,Chronic_Ischemic_Heart_Disease,Chronic_Osteoporosis,Chronic_Rheumatoid_Arthritis_Osteoarthritis,Chronic_Stroke_TIA,Inpatient_Medical_Reimbursement,Inpatient_Beneficiary_Responsibility,Inpatient_Primary_Payer_Reimbursement,Outpatient_Medical_Reimbursement,Outpatient_Beneficiary_Responsibility,Outpatient_Primary_Payer_Reimbursement,Carrier_Medical_Reimbursement,Carrier_Beneficiary_Responsibility,Carrier_Primary_Payer_Reimbursement
0,000002F7E0A96C32,19190701,NaN,2,2,0,5,400,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00001C24EE7B06AC,19360501,NaN,1,1,0,11,530,12,12,0,12,2,2,2,2,2,2,2,1,2,2,2,0.0,0.0,0.0,200.0,40.0,0.0,800.0,260.0,0.0
2,000072CF62193213,19310401,NaN,2,1,0,34,120,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,130.0,70.0,0.0,440.0,30.0,50.0
3,0000DCD33779ED8A,19420501,NaN,2,2,0,11,190,12,12,0,12,1,1,2,1,2,2,2,2,2,2,2,0.0,0.0,0.0,90.0,20.0,0.0,930.0,200.0,0.0
4,0000F1EB530967F3,19350401,NaN,2,1,0,23,720,12,12,0,12,1,1,2,1,2,1,1,1,1,1,2,0.0,0.0,0.0,70.0,200.0,0.0,4950.0,1340.0,0.0


In [5]:
# Define a dictionary for renaming columns
outpatient_rename_dict = {
    'DESYNPUF_ID': 'Beneficiary_ID',
    'CLM_ID': 'Claim_ID',
    'SEGMENT': 'Claim_Line_Segment',
    'CLM_FROM_DT': 'Claim_Start_Date',
    'CLM_THRU_DT': 'Claim_End_Date',
    'PRVDR_NUM': 'Provider_Institution',
    'CLM_PMT_AMT': 'Claim_Payment_Amount',
    'NCH_PRMRY_PYR_CLM_PD_AMT': 'Primary_Payer_Claim_Paid_Amount',
    'AT_PHYSN_NPI': 'Attending_Physician_NPI',
    'OP_PHYSN_NPI': 'Operating_Physician_NPI',
    'OT_PHYSN_NPI': 'Other_Physician_NPI',
    'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM': 'Beneficiary_Blood_Deductible_Liability_Amount',
    'ICD9_DGNS_CD_1': 'Diagnosis_Code_1',
    'ICD9_DGNS_CD_2': 'Diagnosis_Code_2',
    'ICD9_DGNS_CD_3': 'Diagnosis_Code_3',
    'ICD9_DGNS_CD_4': 'Diagnosis_Code_4',
    'ICD9_DGNS_CD_5': 'Diagnosis_Code_5',
    'ICD9_DGNS_CD_6': 'Diagnosis_Code_6',
    'ICD9_DGNS_CD_7': 'Diagnosis_Code_7',
    'ICD9_DGNS_CD_8': 'Diagnosis_Code_8',
    'ICD9_DGNS_CD_9': 'Diagnosis_Code_9',
    'ICD9_DGNS_CD_10': 'Diagnosis_Code_10',
    'ICD9_PRCDR_CD_1': 'Procedure_Code_1',
    'ICD9_PRCDR_CD_2': 'Procedure_Code_2',
    'ICD9_PRCDR_CD_3': 'Procedure_Code_3',
    'ICD9_PRCDR_CD_4': 'Procedure_Code_4',
    'ICD9_PRCDR_CD_5': 'Procedure_Code_5',
    'ICD9_PRCDR_CD_6': 'Procedure_Code_6',
    'NCH_BENE_PTB_DDCTBL_AMT': 'Beneficiary_Part_B_Deductible_Amount',
    'NCH_BENE_PTB_COINSRNC_AMT': 'Beneficiary_Part_B_Coinsurance_Amount',
    'ADMTNG_ICD9_DGNS_CD': 'Admitting_Diagnosis_Code',
    'HCPCS_CD_1': 'HCPCS_Code_1',
    'HCPCS_CD_2': 'HCPCS_Code_2',
    'HCPCS_CD_3': 'HCPCS_Code_3',
    'HCPCS_CD_4': 'HCPCS_Code_4',
    'HCPCS_CD_5': 'HCPCS_Code_5',
    'HCPCS_CD_45': 'HCPCS_Code_45'
}

# Rename columns using the dictionary
outpatient_claims_df.rename(columns=outpatient_rename_dict, inplace=True)

In [6]:
outpatient_claims_df.head()

,Beneficiary_ID,Claim_ID,Claim_Line_Segment,Claim_Start_Date,Claim_End_Date,Provider_Institution,Claim_Payment_Amount,Primary_Payer_Claim_Paid_Amount,Attending_Physician_NPI,Operating_Physician_NPI,Other_Physician_NPI,Beneficiary_Blood_Deductible_Liability_Amount,Diagnosis_Code_1,Diagnosis_Code_2,Diagnosis_Code_3,Diagnosis_Code_4,Diagnosis_Code_5,Diagnosis_Code_6,Diagnosis_Code_7,Diagnosis_Code_8,Diagnosis_Code_9,Diagnosis_Code_10,Procedure_Code_1,Procedure_Code_2,Procedure_Code_3,Procedure_Code_4,Procedure_Code_5,Procedure_Code_6,Beneficiary_Part_B_Deductible_Amount,Beneficiary_Part_B_Coinsurance_Amount,Admitting_Diagnosis_Code,HCPCS_Code_1,HCPCS_Code_2,HCPCS_Code_3,HCPCS_Code_4,HCPCS_Code_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_Code_45
0,00001C24EE7B06AC,684562269783396,1,20090404.0,20090404.0,1100SK,200.0,0.0,1.298827e+09,NaN,NaN,0.0,74560,V5861,42731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,40.0,NaN,93303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00001C24EE7B06AC,684012269893042,1,20100310.0,20100310.0,1100SK,500.0,0.0,1.298827e+09,NaN,NaN,0.0,7879,V4589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,74000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000072CF62193213,684012269540703,1,20080130.0,20080130.0,1000AH,50.0,0.0,8.929521e+09,NaN,8.523329e+09,0.0,4019,73300,79431,V5869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,80053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000072CF62193213,684472269696971,1,20080301.0,20080301.0,1000AH,70.0,0.0,8.382688e+09,NaN,NaN,0.0,4019,71699,V7644,78605,7226,V1582,43882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0,NaN,36415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000072CF62193213,684122269778934,1,20080322.0,20080322.0,3400HK,40.0,0.0,4.404237e+09,NaN,NaN,0.0,1736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,1733,88305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Shape of the dataset

In [7]:
beneficiary_df.shape

(114641, 32)

In [8]:
outpatient_claims_df.shape

(790044, 76)

## 3. Data Preprocessing

#### Data Checks to perform

- Check Missing values
- Check Duplicates
- Check the number of unique values of each column
- Check data type
- Check statistics of data set
- Check various categories present in the different categorical column

### 3.1. Beneficiary Dataset

#### Check Missing values

In [9]:
beneficiary_df.isna().sum()

Beneficiary_ID                                      0
Date_of_Birth                                       0
Date_of_Death                                  112811
Sex                                                 0
Race_Code                                           0
ESRD_Indicator                                      0
State_Code                                          0
County_Code                                         0
Coverage_Part_A_Months                              0
Coverage_Part_B_Months                              0
Coverage_HMO_Months                                 0
Coverage_Part_D_Plan_Months                         0
Chronic_Alzheimer_Disease                           0
Chronic_Heart_Failure                               0
Chronic_Kidney_Disease                              0
Chronic_Cancer                                      0
Chronic_Obstructive_Pulmonary_Disease               0
Chronic_Depression                                  0
Chronic_Diabetes            

In [10]:
beneficiary_df[~beneficiary_df.Date_of_Death.isna()]

,Beneficiary_ID,Date_of_Birth,Date_of_Death,Sex,Race_Code,ESRD_Indicator,State_Code,County_Code,Coverage_Part_A_Months,Coverage_Part_B_Months,Coverage_HMO_Months,Coverage_Part_D_Plan_Months,Chronic_Alzheimer_Disease,Chronic_Heart_Failure,Chronic_Kidney_Disease,Chronic_Cancer,Chronic_Obstructive_Pulmonary_Disease,Chronic_Depression,Chronic_Diabetes,Chronic_Ischemic_Heart_Disease,Chronic_Osteoporosis,Chronic_Rheumatoid_Arthritis_Osteoarthritis,Chronic_Stroke_TIA,Inpatient_Medical_Reimbursement,Inpatient_Beneficiary_Responsibility,Inpatient_Primary_Payer_Reimbursement,Outpatient_Medical_Reimbursement,Outpatient_Beneficiary_Responsibility,Outpatient_Primary_Payer_Reimbursement,Carrier_Medical_Reimbursement,Carrier_Beneficiary_Responsibility,Carrier_Primary_Payer_Reimbursement
140,0050D66259ADF7AC,19210701,20090101.0,2,1,0,18,0,12,12,0,12,2,2,2,2,2,1,1,1,1,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,00539E095CB3CD42,19311201,20091101.0,2,1,0,46,170,12,12,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,00C6A75C4B231127,19390901,20090601.0,2,1,Y,52,700,12,12,12,12,2,1,1,2,1,2,1,1,2,2,2,14000.0,1068.0,0.0,7100.0,2020.0,0.0,5730.0,1990.0,0.0
381,00D401552AA18A58,19370201,20090601.0,1,3,0,24,610,12,12,12,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
385,00D6FE25DB4A36CB,19370901,20091001.0,2,1,0,36,330,12,12,0,12,2,1,2,2,2,2,1,1,2,2,2,0.0,0.0,0.0,1130.0,490.0,0.0,1400.0,420.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114416,FF8A52B9A6BE4CA0,19340901,20090901.0,1,1,0,34,360,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114495,FFB3838B91E7832A,19560301,20090101.0,1,1,0,37,130,12,12,0,12,2,1,2,2,2,1,2,1,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114529,FFC2ED8A74433F28,19340201,20090201.0,2,5,0,54,999,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,80.0,20.0,0.0
114544,FFC8FE59ED3FCAA4,19350901,20090901.0,2,1,Y,26,630,12,12,12,12,2,1,2,2,2,2,2,1,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,1630.0,440.0,0.0


- **Notes:**
    - Missing values: There are no missing values in the data set except date of death as expected
        - TODO: Rows with date of death filled, what can be done with this data?

#### Check Duplicates

In [11]:
beneficiary_df.duplicated().sum()

np.int64(0)

- **Notes:**
    - Duplicates: There are no duplicate rows

#### Checking the number of unique values of each column

In [12]:
beneficiary_df.nunique()

Beneficiary_ID                                 114641
Date_of_Birth                                     900
Date_of_Death                                      12
Sex                                                 2
Race_Code                                           4
ESRD_Indicator                                      2
State_Code                                         52
County_Code                                       310
Coverage_Part_A_Months                             13
Coverage_Part_B_Months                             13
Coverage_HMO_Months                                13
Coverage_Part_D_Plan_Months                        13
Chronic_Alzheimer_Disease                           2
Chronic_Heart_Failure                               2
Chronic_Kidney_Disease                              2
Chronic_Cancer                                      2
Chronic_Obstructive_Pulmonary_Disease               2
Chronic_Depression                                  2
Chronic_Diabetes            

- **Notes:**
    - Data includes:
        - Demographic data
        - Coverage details
        - Chronic conditions
        - Reimbursement information
    - TODO: Plots connectign Chronic kidney disease and ESRD Indicator
    - TODO: Plots check correlation between chronic disease features before combining them
    - TODO: Chronic conditions are time varying could be time series if other beneficiary data is provided to add in notes

#### Check data types

In [13]:
# Check Null and Dtypes
beneficiary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114641 entries, 0 to 114640
Data columns (total 32 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Beneficiary_ID                               114641 non-null  object 
 1   Date_of_Birth                                114641 non-null  int64  
 2   Date_of_Death                                1830 non-null    float64
 3   Sex                                          114641 non-null  int64  
 4   Race_Code                                    114641 non-null  int64  
 5   ESRD_Indicator                               114641 non-null  object 
 6   State_Code                                   114641 non-null  int64  
 7   County_Code                                  114641 non-null  int64  
 8   Coverage_Part_A_Months                       114641 non-null  int64  
 9   Coverage_Part_B_Months                       114641 non-nul

In [14]:
# Define which columns to convert to categorical
categorical_columns = [
    'Sex', 'Race_Code', 'ESRD_Indicator', 'State_Code', 'County_Code',
    'Chronic_Alzheimer_Disease', 'Chronic_Heart_Failure', 'Chronic_Kidney_Disease', 
    'Chronic_Cancer', 'Chronic_Obstructive_Pulmonary_Disease', 'Chronic_Depression', 
    'Chronic_Diabetes', 'Chronic_Ischemic_Heart_Disease', 'Chronic_Osteoporosis', 
    'Chronic_Rheumatoid_Arthritis_Osteoarthritis', 'Chronic_Stroke_TIA'
]

# Convert columns to categorical data types
beneficiary_df[categorical_columns] = beneficiary_df[categorical_columns].astype('category')

#### Date Columns

In [15]:
# Convert 'Date_of_Birth' to datetime format using the specific format YYYYMMDD
beneficiary_df['Date_of_Birth'] = pd.to_datetime(beneficiary_df['Date_of_Birth'], format='%Y%m%d', errors='coerce')

# Convert 'Date_of_Death' to datetime format (assuming it's also in a similar format)
beneficiary_df['Date_of_Death'] = pd.to_datetime(beneficiary_df['Date_of_Death'], format='%Y%m%d', errors='coerce')

# Check unique birth years after conversion
# print(beneficiary_df['Date_of_Birth'].dt.year.unique())

In [16]:
# Define reference year
reference_year = 2009

In [17]:
# Calculate Age as of the reference year
beneficiary_df['Age_as_of_2009'] = reference_year - beneficiary_df['Date_of_Birth'].dt.year

# Calculate Age at Death: TODO CHECK IF THIS NEEDED FOR ANALYSIS
# beneficiary_df['Age_at_Death'] = beneficiary_df['Date_of_Death'].dt.year - beneficiary_df['Date_of_Birth'].dt.year

# Create a single 'Age' column
#beneficiary_df['Age'] = beneficiary_df.apply(
#    lambda row: row['Age_at_Death'] if pd.notna(row['Date_of_Death']) else row['Age_as_of_2009'],
#    axis=1
#)

In [18]:
# Extract year from 'Date_of_Birth'
beneficiary_df['Year_of_Birth'] = beneficiary_df['Date_of_Birth'].dt.year

# Function to categorize year of birth
def categorize_year_of_birth(year):
    if year < 1924:
        return 'Pre-1924'
    elif 1924 <= year <= 1928:
        return '1924–1928'
    elif 1929 <= year <= 1933:
        return '1929–1933'
    elif 1934 <= year <= 1938:
        return '1934–1938'
    elif 1939 <= year <= 1943:
        return '1939–1943'
    else:
        return 'Post-1943'

# Apply the function to create a new column
beneficiary_df['Year_of_Birth_Category'] = beneficiary_df['Year_of_Birth'].apply(categorize_year_of_birth)

In [19]:
# Drop intermediate columns if no longer needed: TODO
beneficiary_df = beneficiary_df.drop(columns=['Date_of_Birth', 'Date_of_Death'])

# Drop intermediate columns if no longer needed: TODO
# beneficiary_df = beneficiary_df.drop(columns=['Age_as_of_2009', 'Age_at_Death'])

# Drop intermediate columns: TODO
# beneficiary_df = beneficiary_df.drop(columns=['Year_of_Birth'])

In [20]:
# Counter(beneficiary_df.Year_of_Birth_Category)

#### Reassign Categories

In [21]:
# Reassign 'Sex' categories
beneficiary_df['Sex'] = beneficiary_df['Sex'].map({1: 'Male', 2: 'Female'})

# Check the distribution after mapping
Counter(beneficiary_df.Sex)

Counter({'Female': 63545, 'Male': 51096})

In [22]:
# Verifying Race distribution
# Counter(beneficiary_df.Race_Code)
# (4916+2630)/(94945+12141+4916+2639)

In [23]:
# Reassign the 'Race_Code' categories
race_mapping = {1: 'White', 2: 'Black', 3: 'Other/Hispanic', 4: 'Other/Hispanic'}

# Apply the mapping to the Race_Code column
beneficiary_df['Race_Code'] = beneficiary_df['Race_Code'].map(race_mapping)

# Check the result
Counter(beneficiary_df['Race_Code'])


Counter({'White': 94945, 'Black': 12141, 'Other/Hispanic': 4916, nan: 2639})

In [24]:
beneficiary_df.head()

,Beneficiary_ID,Sex,Race_Code,ESRD_Indicator,State_Code,County_Code,Coverage_Part_A_Months,Coverage_Part_B_Months,Coverage_HMO_Months,Coverage_Part_D_Plan_Months,Chronic_Alzheimer_Disease,Chronic_Heart_Failure,Chronic_Kidney_Disease,Chronic_Cancer,Chronic_Obstructive_Pulmonary_Disease,Chronic_Depression,Chronic_Diabetes,Chronic_Ischemic_Heart_Disease,Chronic_Osteoporosis,Chronic_Rheumatoid_Arthritis_Osteoarthritis,Chronic_Stroke_TIA,Inpatient_Medical_Reimbursement,Inpatient_Beneficiary_Responsibility,Inpatient_Primary_Payer_Reimbursement,Outpatient_Medical_Reimbursement,Outpatient_Beneficiary_Responsibility,Outpatient_Primary_Payer_Reimbursement,Carrier_Medical_Reimbursement,Carrier_Beneficiary_Responsibility,Carrier_Primary_Payer_Reimbursement,Age_as_of_2009,Year_of_Birth,Year_of_Birth_Category
0,000002F7E0A96C32,Female,Black,0,5,400,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90,1919,Pre-1924
1,00001C24EE7B06AC,Male,White,0,11,530,12,12,0,12,2,2,2,2,2,2,2,1,2,2,2,0.0,0.0,0.0,200.0,40.0,0.0,800.0,260.0,0.0,73,1936,1934–1938
2,000072CF62193213,Female,White,0,34,120,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,130.0,70.0,0.0,440.0,30.0,50.0,78,1931,1929–1933
3,0000DCD33779ED8A,Female,Black,0,11,190,12,12,0,12,1,1,2,1,2,2,2,2,2,2,2,0.0,0.0,0.0,90.0,20.0,0.0,930.0,200.0,0.0,67,1942,1939–1943
4,0000F1EB530967F3,Female,White,0,23,720,12,12,0,12,1,1,2,1,2,1,1,1,1,1,2,0.0,0.0,0.0,70.0,200.0,0.0,4950.0,1340.0,0.0,74,1935,1934–1938


- **Notes:**
    - Assumptions: TODO reframe if needed
    - Based on the distribution of Sex(Male 44.4, Female 55.6) from the user guide reassigning categories to Sex

### 3.5 Check statistics of data set

In [106]:
beneficiary_df.head()

,Beneficiary_ID,Sex,Race_Code,ESRD_Indicator,State_Code,County_Code,Coverage_Part_A_Months,Coverage_Part_B_Months,Coverage_HMO_Months,Coverage_Part_D_Plan_Months,Chronic_Alzheimer_Disease,Chronic_Heart_Failure,Chronic_Kidney_Disease,Chronic_Cancer,Chronic_Obstructive_Pulmonary_Disease,Chronic_Depression,Chronic_Diabetes,Chronic_Ischemic_Heart_Disease,Chronic_Osteoporosis,Chronic_Rheumatoid_Arthritis_Osteoarthritis,Chronic_Stroke_TIA,Inpatient_Medical_Reimbursement,Inpatient_Beneficiary_Responsibility,Inpatient_Primary_Payer_Reimbursement,Outpatient_Medical_Reimbursement,Outpatient_Beneficiary_Responsibility,Outpatient_Primary_Payer_Reimbursement,Carrier_Medical_Reimbursement,Carrier_Beneficiary_Responsibility,Carrier_Primary_Payer_Reimbursement,Age_as_of_2009,Year_of_Birth,Year_of_Birth_Category
0,000002F7E0A96C32,2,2,0,5,400,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90,1919,Pre-1924
1,00001C24EE7B06AC,1,1,0,11,530,12,12,0,12,2,2,2,2,2,2,2,1,2,2,2,0.0,0.0,0.0,200.0,40.0,0.0,800.0,260.0,0.0,73,1936,1934–1938
2,000072CF62193213,2,1,0,34,120,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,130.0,70.0,0.0,440.0,30.0,50.0,78,1931,1929–1933
3,0000DCD33779ED8A,2,2,0,11,190,12,12,0,12,1,1,2,1,2,2,2,2,2,2,2,0.0,0.0,0.0,90.0,20.0,0.0,930.0,200.0,0.0,67,1942,1939–1943
4,0000F1EB530967F3,2,1,0,23,720,12,12,0,12,1,1,2,1,2,1,1,1,1,1,2,0.0,0.0,0.0,70.0,200.0,0.0,4950.0,1340.0,0.0,74,1935,1934–1938


In [41]:
Counter(beneficiary_df.Sex)

Counter({2: 63545, 1: 51096})

In [42]:
Counter(beneficiary_df.Race_Code)

Counter({1: 94945, 2: 12141, 3: 4916, 5: 2639})

In [40]:
outpatient_claims_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790044 entries, 0 to 790043
Data columns (total 76 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Beneficiary_ID                                 790044 non-null  object 
 1   Claim_ID                                       790044 non-null  int64  
 2   Claim_Line_Segment                             790044 non-null  int64  
 3   Claim_Start_Date                               779016 non-null  float64
 4   Claim_End_Date                                 779016 non-null  float64
 5   Provider_Institution                           790044 non-null  object 
 6   Claim_Payment_Amount                           790044 non-null  float64
 7   Primary_Payer_Claim_Paid_Amount                790044 non-null  float64
 8   Attending_Physician_NPI                        772604 non-null  float64
 9   Operating_Physician_NPI              

In [33]:
beneficiary_df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date_of_Birth,114641,1970-01-01 00:00:00.019364038,1970-01-01 00:00:00.019090101,1970-01-01 00:00:00.019281101,1970-01-01 00:00:00.019360501,1970-01-01 00:00:00.019420301,1970-01-01 00:00:00.019831201,NaN
Date_of_Death,1830,1970-01-01 00:00:00.020090634,1970-01-01 00:00:00.020090101,1970-01-01 00:00:00.020090301,1970-01-01 00:00:00.020090601,1970-01-01 00:00:00.020090901,1970-01-01 00:00:00.020091201,NaN
Coverage_Part_A_Months,114641.0,11.228653,0.0,12.0,12.0,12.0,12.0,2.926545
Coverage_Part_B_Months,114641.0,11.113284,0.0,12.0,12.0,12.0,12.0,3.093597
Coverage_HMO_Months,114641.0,3.332289,0.0,0.0,0.0,12.0,12.0,5.303927
Coverage_Part_D_Plan_Months,114641.0,9.070237,0.0,6.0,12.0,12.0,12.0,4.869408
Inpatient_Medical_Reimbursement,114641.0,2158.44,-8000.0,0.0,0.0,0.0,136000.0,7167.900814
Inpatient_Beneficiary_Responsibility,114641.0,247.422057,0.0,0.0,0.0,0.0,39340.0,781.243963
Inpatient_Primary_Payer_Reimbursement,114641.0,101.529994,0.0,0.0,0.0,0.0,86000.0,1951.358679
Outpatient_Medical_Reimbursement,114641.0,762.027111,-90.0,0.0,120.0,800.0,47010.0,1876.649919


#### Initial Observations

- **Data includes:**
    - Demographic data
    - Coverage details
    - Chronic conditions
    - Reimbursement information

- **Coverage Months:**
    - Part A/B: Most beneficiaries have 12 months of coverage, with some variation.
    - HMO: Significant variability; many have no coverage.
    - Part D: Most beneficiaries have 12 months, with substantial variability.

- **Reimbursement and Responsibility:** High variability in reimbursements and responsibilities, especially in inpatient and outpatient categories, with a few extreme values.
    - Inpatient Medical Reimbursement:
        - High mean `$2158.44` but extremely high standard deviation `$7167.90`, indicating a few very high values are **skewing** the data.
    - Inpatient Beneficiary Responsibility:
        - Mean responsibility is `$247.42` with high variability (standard deviation of `$781.24`), showing considerable differences in what beneficiaries owe.
    - Inpatient Primary Payer Reimbursement:
        - Average reimbursement is `$101.53` with a large standard deviation `$1951.36`, suggesting a few extreme outliers.
    - Outpatient Medical Reimbursement:
        - Mean is `$762.03` with a high standard deviation `$1876.65`, reflecting a mix of mostly lower and a few very high reimbursements.
    - Outpatient Beneficiary Responsibility:
        - Average responsibility is `$234.91` with significant variability (standard deviation of `$539.39`).
    - Outpatient Primary Payer Reimbursement:
        - Mean is `$30.71` with moderate standard deviation (`$414.68`), indicating some very high reimbursements.
    - Carrier Medical Reimbursement:
        - Mean is `$1337.59` with a high standard deviation (`$1525.17`), highlighting wide variability in reimbursement amounts.
    - Carrier Beneficiary Responsibility:
        - Average is `$374.56` with a high standard deviation (`$423.54`), showing considerable variability in what beneficiaries are responsible for.
    - Carrier Primary Payer Reimbursement:
        - Mean is `$21.11` with a standard deviation of `$96.01`, indicating many low values with a few higher amounts.

- **Age:**
    - Average age is around 38 years with moderate variation.
 
  - TODO: Find inconsistencies with the outpatient columns from beneficiary with outpatient df columns
 
      - Remember age is 39 seems weird
   

#### Actions to be taken:

- TODO: Check for outliers in skewed data

## 3.2. EDA - Claims Dataset Information

In [23]:
outpatient_claims_df.head()

,Beneficiary_ID,Claim_ID,Claim_Line_Segment,Claim_Start_Date,Claim_End_Date,Provider_Institution,Claim_Payment_Amount,Primary_Payer_Claim_Paid_Amount,Attending_Physician_NPI,Operating_Physician_NPI,Other_Physician_NPI,Beneficiary_Blood_Deductible_Liability_Amount,Diagnosis_Code_1,Diagnosis_Code_2,Diagnosis_Code_3,Diagnosis_Code_4,Diagnosis_Code_5,Diagnosis_Code_6,Diagnosis_Code_7,Diagnosis_Code_8,Diagnosis_Code_9,Diagnosis_Code_10,Procedure_Code_1,Procedure_Code_2,Procedure_Code_3,Procedure_Code_4,Procedure_Code_5,Procedure_Code_6,Beneficiary_Part_B_Deductible_Amount,Beneficiary_Part_B_Coinsurance_Amount,Admitting_Diagnosis_Code,HCPCS_Code_1,HCPCS_Code_2,HCPCS_Code_3,HCPCS_Code_4,HCPCS_Code_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_Code_45
0,00001C24EE7B06AC,684562269783396,1,20090404.0,20090404.0,1100SK,200.0,0.0,1.298827e+09,NaN,NaN,0.0,74560,V5861,42731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,40.0,NaN,93303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00001C24EE7B06AC,684012269893042,1,20100310.0,20100310.0,1100SK,500.0,0.0,1.298827e+09,NaN,NaN,0.0,7879,V4589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,74000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000072CF62193213,684012269540703,1,20080130.0,20080130.0,1000AH,50.0,0.0,8.929521e+09,NaN,8.523329e+09,0.0,4019,73300,79431,V5869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,80053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000072CF62193213,684472269696971,1,20080301.0,20080301.0,1000AH,70.0,0.0,8.382688e+09,NaN,NaN,0.0,4019,71699,V7644,78605,7226,V1582,43882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0,NaN,36415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000072CF62193213,684122269778934,1,20080322.0,20080322.0,3400HK,40.0,0.0,4.404237e+09,NaN,NaN,0.0,1736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,1733,88305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
